In [37]:
# this part of the codes fetches requirements
import hashlib
from phe import paillier
import random as rd
import numpy as np

In [38]:
listOfIDS = []
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
dictionaryOfRandomNumbers = {}
dictionaryOfBids = {}
finalSubmittedDataFirstPhase = [] #will be sused to stored the final resuklt of the two bidder for bid submission
highestbidder = 0 
numberOfBidders = 0 
publicKeyAuc, privateKeyAuc = 0, 0
finalDataOfSecondPhase = []
productOfAllRandomNumbers = 1

In [39]:
#  this part of the code creates unique BID for all participants
def createBiddersAndKeys(n):
  global numberOfBidders,dictionaryOfPublicKeys,dictionaryOfPrivateKeys,publicKeyAuc,privateKeyAuc
  numberOfBidders = n
  for i in range(0,numberOfBidders):
    i = str(i)
    listOfIDS.append(hashlib.sha256(i.encode('utf-8')).hexdigest())
  i = 0
  publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 16)
  for i in range(0, numberOfBidders):
    x,y = paillier.generate_paillier_keypair(n_length= 32)
    dictionaryOfPublicKeys[listOfIDS[i]],dictionaryOfPrivateKeys[listOfIDS[i]]  = x ,y
  i = 0

end of the Registration phase and start of Bid Submission Phase

In [40]:
# this part of the code generate random number and distinct bid values for all bidders and Auctioneer
def  createRandomNumberAndBids():
    global dictionaryOfBids,dictionaryOfRandomNumbers
    bids  = rd.sample(range(1, numberOfBidders+1 ), numberOfBidders)
    for i in range(0,numberOfBidders):
        r  = rd.randint(100,10000)
        dictionaryOfRandomNumbers[listOfIDS[i]] =  r
        dictionaryOfBids[listOfIDS[i]] = bids[i]
    i = 0
    print(dictionaryOfBids)
    print(dictionaryOfRandomNumbers)


In [41]:
# this PART OF THE CODE SIMULATES BIDDER GENERATING PRODUCT AND THAN SENDING TO NEXT BIDDER

def initFirstPhase():
    global finalSubmittedDataFirstPhase
    for i in range(0,numberOfBidders):
        id = listOfIDS[i]
        dataPerBidder= []
        encrytionOfBidAndRandomNumber = publicKeyAuc.raw_encrypt(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
        # this part of code is forwarding the encrytionOfBidAndRandomNumber to all other bidders and after multiplication of the random number of the next
        # bidder it is sent to the Auctioneer immediately
        for j in range(0,numberOfBidders):
            if j != i:
                idOfNextBidder = listOfIDS[j]
                pubKeyNextBidder, priKeyNextBidder= dictionaryOfPublicKeys[idOfNextBidder],dictionaryOfPrivateKeys[idOfNextBidder]
                e = pubKeyNextBidder.raw_encrypt(encrytionOfBidAndRandomNumber)
                e = priKeyNextBidder.raw_decrypt(e)
                encrytionOfBidAndBothRandomNumber = e ** dictionaryOfRandomNumbers[idOfNextBidder]
                # this part of the code simulates the storing of the data of the bid summision to auctioneer data
                dataPerBidder.append(privateKeyAuc.raw_decrypt(encrytionOfBidAndBothRandomNumber))
            else:
                dataPerBidder.append(0)
        finalSubmittedDataFirstPhase.append(dataPerBidder)
    print(finalSubmittedDataFirstPhase)
    i,j = 0,0

implementation of the second phase here auctioneer reforward each packet to every bidder to multiply their random number.

In [42]:
# #this function works as the intermediate steps where we are requesting each bidder to multiplyt his random number to the given ciphertext
# def productOfRandomNumbers():
#     global productOfAllRandomNumbers
#     for k in range(numberOfBidders):
#         productOfAllRandomNumbers = productOfAllRandomNumbers * dictionaryOfRandomNumbers[listOfIDS[k]]


In [43]:
def everyBiddersRandomNumberMultiplication(encryptedAuctioneerRandomNumber):
    global productOfAllRandomNumbers
    print("////")
    output = encryptedAuctioneerRandomNumber
    for i in range(numberOfBidders):     
        print("zz")
        output  = pow(output,dictionaryOfBids[listOfIDS[i]])
        print(output)
    return output

In [44]:
def initSecondPhase():
    global finalDataOfSecondPhase
    auctioneerRandomNumber = rd.randint(2,2)
    print(auctioneerRandomNumber)
    for i in range(len(finalSubmittedDataFirstPhase)):
        dataPerBidder = []
        for j in range(numberOfBidders):
            if finalSubmittedDataFirstPhase[i][j] == 0 :
                dataPerBidder.append(0)
            else:
                encryptedValueByAuctioneer = publicKeyAuc.raw_encrypt(auctioneerRandomNumber)
                returnValue = everyBiddersRandomNumberMultiplication(encryptedValueByAuctioneer)
                dataPerBidder.append((privateKeyAuc.raw_decrypt(returnValue)))
        finalDataOfSecondPhase.append(dataPerBidder)
    i,j = 0,0


In [45]:
# def runSimulation():
#     print("//")
#     createBiddersAndKeys(2)
#     print("//")
#     createRandomNumberAndBids()
#     print("//")
#     initFirstPhase()
#     print("//")
#     initSecondPhase()
#     print("//")

In [46]:
createBiddersAndKeys(10)
createRandomNumberAndBids()



{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 3, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 9, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 5, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce': 1, '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a': 7, 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d': 10, 'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683': 4, '7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451': 8, '2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3': 6, '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7': 2}
{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 7228, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 9029, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 639, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce'

In [47]:
initFirstPhase()

[[0, 33952, 4139, 9571, 38998, 8156, 33276, 36576, 4304, 31323], [15010, 0, 35294, 25276, 36891, 10283, 19593, 38448, 25381, 17711], [35847, 14783, 0, 32812, 32504, 43255, 14371, 21778, 24977, 16264], [32139, 12458, 15247, 0, 13977, 27170, 2582, 12766, 41810, 20556], [33098, 28693, 36821, 10993, 0, 26962, 34849, 41797, 2430, 17947], [41661, 1776, 43087, 11162, 13704, 0, 25478, 18202, 25354, 41153], [945, 8708, 28866, 10328, 26117, 36245, 0, 1032, 25537, 41486], [10690, 34176, 8791, 15282, 4345, 5877, 2064, 0, 7893, 7956], [8608, 31395, 38657, 33745, 26896, 23897, 16594, 38487, 0, 21046], [20882, 42534, 41244, 41112, 11331, 42969, 20743, 1989, 35964, 0]]


In [49]:
initSecondPhase()

2
////
zz
2171187052000083655225565032
zz
1072198421326902640274405132177006987255172603317464520603019977093165360096988384703513490570575131136292380614754522357708865104380881095943211601535352762263997370184916901049093905003146376726911036547451546390354328630535474388656470813048832
zz
14170194690079997150791180003967046544855853626366847500652018920232953536328106370468759964988459462008751778162328428031004388121717800794160394365362804637353634648741754096544638130265057001319058688972143185648323149185441335214895563143678295188551285643481215570174277619666472603428895734018435581497023363048731319628504871842696567396473443837573178709208572469828914946133861039912686559085485043850745993455914126483328941447699855799648628358423810425539655903028974099299706942711447693733912235203105628790025475679961271605335394898287195946669974626748118158251208710850292208075187293269799962646525264494694396316770276232790973674616559775523658300757680842866534730013704935231904925990

In [ ]:
random = publicKeyAuc.raw_encrypt(2)
output = 1
for i in range(10):
    output  = pow(output,dictionaryOfBids[listOfIDS[i]])
print(privateKeyAuc.raw_decrypt(output))   

In [ ]:

for i in range(len(finalDataOfSecondPhase)):
    print(finalDataOfSecondPhase[i])
i = 0

In [ ]:
for i in range(len(finalSubmittedDataFirstPhase)):
    print(finalSubmittedDataFirstPhase[i])
i = 0

[0, 50947920, 77521920, 94850640, 158751840, 126961920, 179096400, 208488480, 231651120, 129903600]
[25473960, 0, 32316480, 39540285, 66178710, 52926480, 74659725, 86912370, 96568155, 54152775]
[46513152, 38779776, 0, 72196992, 120836352, 96638976, 136321920, 158694144, 176324736, 98878080]
[9485064, 7908057, 12032832, 0, 24641214, 19706832, 27799065, 32361258, 35956527, 20163435]
[47625552, 39707226, 60418176, 73923642, 0, 98950176, 139582170, 162489444, 180541686, 101242830]
[50784768, 42341184, 64425984, 78827328, 131933568, 0, 148841280, 173268096, 192517824, 107958720]
[143277120, 119455560, 181762560, 222392520, 372219120, 297682560, 0, 488834640, 543143160, 304579800]
[145941936, 121677318, 185143168, 226528806, 379142036, 303219168, 427730310, 0, 553245098, 310244690]
[46330224, 38627262, 58774912, 71913054, 120361124, 96258912, 135785790, 158070028, 0, 98489210]
[116913240, 97474995, 148317120, 181470915, 303728490, 242907120, 342652275, 398886030, 443201445, 0]
